# Pre-Processing of Adaptive Run Data

The cells below are examples of the slow analyses (run in TMUX sessions using ipython) which were performed to generate data for the final analyses.

## Long Analyses to be Run in TMUX Sessions

In [ ]:
# Get the 100 blocked data for the convergence analysis for all runs
import a3fe as a3
import pickle
from a3fe.analyse.process_grads import get_time_series_multiwindow_mbar as get_ts
import os

ligs = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}

for lig_dir in ligs.values():
    assert os.path.exists(lig_dir), f"Directory {lig_dir} does not exist"

equil_dgs = {}
for lig in ligs:
    equil_dgs[lig] = {}
    calc = a3.Calculation(base_dir = ligs[lig])
    for leg in calc.legs:
        equil_dgs[lig][str(leg.leg_type)] = {}
        for stage in leg.stages:
            equil_dgs[lig][str(leg.leg_type)][str(stage.stage_type)] = {}
            dgs, times = get_ts(stage.lam_windows, stage.output_dir)
            equil_dgs[lig][str(leg.leg_type)][str(stage.stage_type)]["dgs"] = dgs
            equil_dgs[lig][str(leg.leg_type)][str(stage.stage_type)]["times"] = times
    calc._close_logging_handlers()
    del(calc)

    # Pickle the current data
    with open("final_analysis/equil_dgs.pkl", "wb") as f:
        pickle.dump(equil_dgs, f)

In [ ]:
# Get the costs of each leg
import a3fe as a3
import pickle

ligs = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}

costs = {}
for lig in ligs:
    costs[lig] = {}
    calc = a3.Calculation(base_dir = ligs[lig])
    for leg in calc.legs:
        costs[lig][str(leg.leg_type)] = leg.relative_simulation_cost
    calc._close_logging_handlers()
    del calc

with open("final_analysis/costs.pkl", "wb") as f:
    pickle.dump(costs, f)

In [ ]:
# Get the costs of each leg
import a3fe as a3
import pickle
from typing import List, Tuple, Dict, Callable, Union

ligs = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}
def read_equil_data(equil_data_path: str) -> Tuple[float, List[float], List[float]]:
    """Read the equilibration time, lambda values and free energies from the equilibration data file."""
    with open(equil_data_path, "r") as f:
        equil_data = f.readlines()
    equil_time = float(equil_data[3].split(":")[1].strip().split()[0])
    pvalues_and_times_list = eval(equil_data[1].split(":")[1].strip())
    p_vals = [pvalue for pvalue, _ in pvalues_and_times_list]
    times = [time for _, time in pvalues_and_times_list]
    return equil_time, p_vals, times

# Extract the times and p values from the output text file for each stage for each leg for each system
equil_times = {}
for lig in ligs:
    print(f"Analysing {lig}")
    equil_times[lig] = {}
    calc = a3.Calculation(base_dir = ligs[lig], stream_log_level=logging.CRITICAL)
    for leg in calc.legs:
        equil_times[lig][str(leg.leg_type)] = {}
        for stage in leg.stages:
            equil_times[lig][str(leg.leg_type)][str(stage.stage_type)] = {}
            equil_data_path = stage.output_dir + "/check_equil_multiwindow_paired_t.txt"
            equil_time, p_vals, times = read_equil_data(equil_data_path)
            # Multiply times by costs to get GPU hours
            times = np.array(times) * costs[lig][str(leg.leg_type)]
            equil_time = equil_time * costs[lig][str(leg.leg_type)]
            equil_times[lig][str(leg.leg_type)][str(stage.stage_type)]["equil_time"] = equil_time
            equil_times[lig][str(leg.leg_type)][str(stage.stage_type)]["p_vals"] = p_vals
            equil_times[lig][str(leg.leg_type)][str(stage.stage_type)]["times"] = times
    calc._close_logging_handlers()
    del(calc)

    # Write most recent version of the dictionary to a pickle
    with open("final_analysis/equil_times.pkl", "wb") as f:
        pickle.dump(equil_times, f)


In [ ]:
import a3fe as a3
import pickle
import logging

calc_dirs = {
    "T4L": {"adaptive": "t4l", "non-adaptive": "../non_adaptive_final/t4l_30000ps"},
    "MIF": {"adaptive": "mif_180_anti", "non-adaptive": "../non_adaptive_final/mif_180_anti_30000ps"},
    "MDM2-PIP2": {"adaptive": "mdm2_pip2_short", "non-adaptive": "../non_adaptive_final/mdm2_pip2_short_30000ps"},
    "PDE2A": {"adaptive": "pde2a_p10", "non-adaptive": "../non_adaptive_final/pde2a_p10_30000"},
    "MDM2-Nutlin": {"adaptive": "mdm2_nutlin_notprot", "non-adaptive": "../non_adaptive_final/mdm2_nutlin_notprot_30000ps"},
}


# Get dictionary of final free energy changes for the 30 ns runs
final_dGs_all = {}
for system in calc_dirs:
    final_dGs_all[system] = {}
    for method in calc_dirs[system]:
        print(f"Analysing {system} {method}")
        calc = a3.Calculation(base_dir=calc_dirs[system][method], stream_log_level=logging.CRITICAL)
        if calc._delta_g is None:
            calc.analyse()
        final_dGs_all[system][method] = {}
        final_dGs_all[system][method]["dgs"] = calc._delta_g
        for leg in calc.legs:
            print(f"Analysing {leg.leg_type}")
            final_dGs_all[system][method][str(leg.leg_type)] = {}
            final_dGs_all[system][method][str(leg.leg_type)]["dg"] = leg._delta_g
            for stage in leg.stages:
                final_dGs_all[system][method][str(leg.leg_type)][str(stage.stage_type)] = {}
                final_dGs_all[system][method][str(leg.leg_type)][str(stage.stage_type)]["dg"] = stage._delta_g
        calc._dump()
        calc._close_logging_handlers()
        del(calc)

        with open("final_analysis/final_dGs_all.pkl", "wb") as f:
            pickle.dump(final_dGs_all, f)


In [ ]:
import a3fe as a3
import pickle
import logging

ligs = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}

lam_vals = {}
for lig in ligs:
    print(f"Analysing {lig}")
    lam_vals[lig] = {}
    calc = a3.Calculation(base_dir = ligs[lig], stream_log_level=logging.CRITICAL)
    for leg in calc.legs:
        lam_vals[lig][str(leg.leg_type)] = {}
        for stage in leg.stages:
            lam_vals[lig][str(leg.leg_type)][str(stage.stage_type)] = stage.lam_vals
    calc._close_logging_handlers()
    del(calc)

    # Write most recent version of the dictionary to a pickle
    with open("final_analysis/lam_vals.pkl", "wb") as f:
        pickle.dump(lam_vals, f)

In [ ]:
import a3fe as a3
import pickle
import logging

ligs = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}

sampling_times = {}

for lig in ligs:
    print(f"Analysing {lig}")
    sampling_times[lig] = {}
    calc = a3.Calculation(base_dir = ligs[lig], stream_log_level=logging.CRITICAL)
    for leg in calc.legs:
        sampling_times[lig][str(leg.leg_type)] = {}
        for stage in leg.stages:
            sampling_times[lig][str(leg.leg_type)][str(stage.stage_type)] = {}
            sampling_times[lig][str(leg.leg_type)][str(stage.stage_type)]["times"] = [lam.tot_simtime for lam in stage.lam_windows]
            sampling_times[lig][str(leg.leg_type)][str(stage.stage_type)]["times"] = [lam.tot_simtime for lam in stage.lam_windows]
            sampling_times[lig][str(leg.leg_type)][str(stage.stage_type)]["equil_times"] = [lam.equil_time * lam.ensemble_size for lam in stage.lam_windows]
            sampling_times[lig][str(leg.leg_type)][str(stage.stage_type)]["lam_vals"] = [lam.lam for lam in stage.lam_windows]

    calc._close_logging_handlers()
    del(calc)

    # Write most recent version of the dictionary to a pickle
    with open("final_analysis/sampling_times.pkl", "wb") as f:
        pickle.dump(sampling_times, f)

In [ ]:
import a3fe as a3
import pickle

adaptive_paths = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}

# Dict mapping directory names to calculation names
non_adapt_paths = {
    "t4l_0.2": "../non_adaptive_final/t4l_200ps",
    "t4l_6": "../non_adaptive_final/t4l_5000ps",
    "t4l_30": "../non_adaptive_final/t4l_30000ps",
    "mif_0.2": "../non_adaptive_final/mif_180_anti_200ps",
    "mif_6": "../non_adaptive_final/mif_180_anti_5000ps",
    "mif_30": "../non_adaptive_final/mif_180_anti_30000ps",
    "mdm2_nut_0.2": "../non_adaptive_final/mdm2_nutlin_notprot_200ps",
    "mdm2_nut_6": "../non_adaptive_final/mdm2_nutlin_notprot_5000ps",
    "mdm2_nut_30": "../non_adaptive_final/mdm2_nutlin_notprot_30000ps",
    "mdm2_pip_0.2": "../non_adaptive_final/mdm2_pip2_short_200ps",
    "mdm2_pip_6": "../non_adaptive_final/mdm2_pip2_short_5000ps",
    "mdm2_pip_30": "../non_adaptive_final/mdm2_pip2_short_30000ps",
    "pde_0.2": "../non_adaptive_final/pde2a_p10_200ps",
    "pde_6": "../non_adaptive_final/pde2a_p10_5000ps",
    "pde_30": "../non_adaptive_final/pde2a_p10_30000",
}

non_adapt_ligs = {
    "T4L": {"0.2 ns": "t4l_0.2", "6 ns": "t4l_6", "30 ns": "t4l_30"},
    "MIF": {"0.2 ns": "mif_0.2", "6 ns": "mif_6", "30 ns": "mif_30"},
    "MDM2-Nutlin": {"0.2 ns": "mdm2_nut_0.2", "6 ns": "mdm2_nut_6", "30 ns": "mdm2_nut_30"},
    "MDM2-PIP2": {"0.2 ns": "mdm2_pip_0.2", "6 ns": "mdm2_pip_6", "30 ns": "mdm2_pip_30"},
    "PDE2A": {"0.2 ns": "pde_0.2", "6 ns": "pde_6", "30 ns": "pde_30"},
}

calc_structure = {"bound":{"restrain":a3.StageType.RESTRAIN, "discharge":a3.StageType.DISCHARGE, "vanish":a3.StageType.VANISH}, "free": {"discharge":a3.StageType.DISCHARGE, "vanish":a3.StageType.VANISH}}

comparitive_conv_data = {}
for lig in adaptive_paths:
    comparitive_conv_data[lig] = {}
    for leg in calc_structure:
        comparitive_conv_data[lig][leg] = {}
        for stage in calc_structure[leg]:
            stage_paths = [
                adaptive_paths[lig] + f"/{leg}/{stage}",
                non_adapt_paths[non_adapt_ligs[lig]["6 ns"]] + f"/{leg}/{stage}",
                non_adapt_paths[non_adapt_ligs[lig]["30 ns"]] + f"/{leg}/{stage}",
            ]
            stage_iterator = a3.run._utils.SimulationRunnerIterator(
                stage_paths,
                a3.Stage, stage_type=calc_structure[leg][stage])
            conv_data = a3.analyse.compare.get_comparitive_convergence_data(
                stage_iterator,
                equilibrated=False,
                mode="block"
            )
            comparitive_conv_data[lig][leg][stage] = {"Adaptive":{}, "6 ns":{}, "30 ns":{}}
            for k, label in enumerate(["Adaptive", "6 ns", "30 ns"]):
                comparitive_conv_data[lig][leg][stage][label]["dgs"] = conv_data[k][0]
                comparitive_conv_data[lig][leg][stage][label]["times"] = conv_data[k][1]

    # Pickle the current data
    with open("final_analysis/comparitive_conv_data.pkl", "wb") as f:
        pickle.dump(comparitive_conv_data, f)


In [ ]:
import a3fe as a3
import pickle

adaptive_paths = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}

# Dict mapping directory names to calculation names
non_adapt_paths = {
    "t4l_0.2": "../non_adaptive_final/t4l_200ps",
    "t4l_6": "../non_adaptive_final/t4l_5000ps",
    "t4l_30": "../non_adaptive_final/t4l_30000ps",
    "mif_0.2": "../non_adaptive_final/mif_180_anti_200ps",
    "mif_6": "../non_adaptive_final/mif_180_anti_5000ps",
    "mif_30": "../non_adaptive_final/mif_180_anti_30000ps",
    "mdm2_nut_0.2": "../non_adaptive_final/mdm2_nutlin_notprot_200ps",
    "mdm2_nut_6": "../non_adaptive_final/mdm2_nutlin_notprot_5000ps",
    "mdm2_nut_30": "../non_adaptive_final/mdm2_nutlin_notprot_30000ps",
    "mdm2_pip_0.2": "../non_adaptive_final/mdm2_pip2_short_200ps",
    "mdm2_pip_6": "../non_adaptive_final/mdm2_pip2_short_5000ps",
    "mdm2_pip_30": "../non_adaptive_final/mdm2_pip2_short_30000ps",
    "pde_0.2": "../non_adaptive_final/pde2a_p10_200ps",
    "pde_6": "../non_adaptive_final/pde2a_p10_5000ps",
    "pde_30": "../non_adaptive_final/pde2a_p10_30000",
}

non_adapt_ligs = {
    "T4L": {"0.2 ns": "t4l_0.2", "6 ns": "t4l_6", "30 ns": "t4l_30"},
    "MIF": {"0.2 ns": "mif_0.2", "6 ns": "mif_6", "30 ns": "mif_30"},
    "MDM2-Nutlin": {"0.2 ns": "mdm2_nut_0.2", "6 ns": "mdm2_nut_6", "30 ns": "mdm2_nut_30"},
    "MDM2-PIP2": {"0.2 ns": "mdm2_pip_0.2", "6 ns": "mdm2_pip_6", "30 ns": "mdm2_pip_30"},
    "PDE2A": {"0.2 ns": "pde_0.2", "6 ns": "pde_6", "30 ns": "pde_30"},
}

calc_structure = {"bound":{"restrain":a3.StageType.RESTRAIN, "discharge":a3.StageType.DISCHARGE, "vanish":a3.StageType.VANISH}, "free": {"discharge":a3.StageType.DISCHARGE, "vanish":a3.StageType.VANISH}}

comparitive_conv_data = {}
for lig in adaptive_paths:
    comparitive_conv_data[lig] = {}
    calc_paths = [
        adaptive_paths[lig],
        non_adapt_paths[non_adapt_ligs[lig]["6 ns"]],
        non_adapt_paths[non_adapt_ligs[lig]["30 ns"]],
    ]
    calc_iterator = a3.run._utils.SimulationRunnerIterator(
        calc_paths,
        a3.Calculation)
    conv_data = a3.analyse.compare.get_comparitive_convergence_data(
        calc_iterator,
        equilibrated=False,
        mode="block"
    )
    comparitive_conv_data[lig] = {"Adaptive":{}, "6 ns":{}, "30 ns":{}}
    for k, label in enumerate(["Adaptive", "6 ns", "30 ns"]):
        comparitive_conv_data[lig][label] = {}
        comparitive_conv_data[lig][label]["dgs"] = conv_data[k][1]
        comparitive_conv_data[lig][label]["times"] = conv_data[k][0]

    # Pickle the current data
    with open("final_analysis/comparitive_conv_data_calcs.pkl", "wb") as f:
        pickle.dump(comparitive_conv_data, f)


In [ ]:
comparitive_conv_data["T4L"]["bound"]["discharge"].keys()

In [ ]:
costs

In [ ]:
import a3fe as a3
import pickle

with open("final_analysis/costs.pkl", "rb") as f:
    costs = pickle.load(f)

with open("final_analysis/comparitive_conv_data.pkl", "rb") as f:
    comparitive_conv_data = pickle.load(f)

# Convert fractions to GPU hours. For the unequilibrated leg, do this by multiplying
# by first converting the fracts to simulation times, then multipling these by the 
# relative simulation cost * the simulation cost reference of 0.21 hr / ns
REF_COST = 0.21 # GPU hours per ns
ligs = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}


for lig in comparitive_conv_data:
    print(f"Converting {lig}")
    for leg in comparitive_conv_data[lig]:
        leg_enum_str = "LegType." + leg.upper()
        cost = REF_COST * costs[lig][leg_enum_str]
        for stage in comparitive_conv_data[lig][leg]:
            for time in comparitive_conv_data[lig][leg][stage]:
                gpu_times = np.array(comparitive_conv_data[lig][leg][stage][time]["times"])*cost
                comparitive_conv_data[lig][leg][stage][time]["gpu_times"] = gpu_times

with open("final_analysis/comparitive_conv_data.pkl", "wb") as f:
    pickle.dump(comparitive_conv_data, f)

In [ ]:
import a3fe as a3
import pickle
import logging

# Dict mapping directory names to calculation names
ligs = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}

dgs_conv = {}

# Code to generate dictionary of blocks of free energy changes for each stage for each leg for each time for each system
dgs_conv[lig] = {}
for time in ligs[lig]:
    dgs_conv[lig][time] = {}
    print(f"Analysing {lig} {time}")
    calc = a3.Calculation(base_dir = ligs[lig], stream_log_level=logging.CRITICAL)
    for leg in calc.legs:
        dgs_conv[lig][time][str(leg.leg_type)] = {}
        for stage in leg.stages:
            dgs_conv[lig][time][str(leg.leg_type)][str(stage.stage_type)] = {}
            fracts = stage._delta_g_convergence_fracts
            fracts, dgs = stage.analyse_convergence(mode="block", equilibrated=False)
            dgs_conv[lig][time][str(leg.leg_type)][str(stage.stage_type)]["fracts"] = fracts
            dgs_conv[lig][time][str(leg.leg_type)][str(stage.stage_type)]["dgs"] = dgs
    calc._close_logging_handlers()
    del(calc)

    # Write most recent version of the dictionary to a pickle
    savename = "final_analysis/dgs_conv_nonequil.pkl" 
    with open(savename, "wb") as f:
        pickle.dump(dgs_conv, f)

In [ ]:
import a3fe as a3
import pickle
import numpy as np

with open("final_analysis/costs.pkl", "rb") as f:
    costs = pickle.load(f)

with open("final_analysis/dgs_conv_nonequil.pkl", "rb") as f:
    dgs_conv_nonequil = pickle.load(f)

# Convert fractions to GPU hours. For the unequilibrated leg, do this by multiplying
# by first converting the fracts to simulation times, then multipling these by the 
# relative simulation cost * the simulation cost reference of 0.21 hr / ns
REF_COST = 0.21 # GPU hours per ns
ligs = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}


# Convert fractions to GPU hours. For the unequilibrated leg, do this by multiplying
# by first converting the fracts to simulation times, then multipling these by the 
# relative simulation cost * the simulation cost reference of 0.21 hr / ns
REF_COST = 0.21 # GPU hours per ns

dg_dicts = {"nonequil": dgs_conv_nonequil}
for dg_type, dg_dict in dg_dicts.items():
    for lig in ligs:
        print(f"Converting {lig} {dg_type}")
        calc = a3.Calculation(base_dir = ligs[lig])
        for leg in calc.legs:
            for stage in leg.stages:
                cost = REF_COST * costs[lig][str(leg.leg_type)]
                start_time = 0 if dg_type == "nonequil" else stage.equil_time * stage.ensemble_size
                end_time = stage.tot_simtime
                fracts = np.array(dg_dict[lig][str(leg.leg_type)][str(stage.stage_type)]["fracts"])
                times = ((end_time - start_time) * fracts ) + start_time
                gpu_times = times * cost
                dg_dict[lig][str(leg.leg_type)][str(stage.stage_type)]["sim_times"] = times
                dg_dict[lig][str(leg.leg_type)][str(stage.stage_type)]["gpu_times"] = gpu_times

        calc._close_logging_handlers()
        del(calc)

with open("final_analysis/dgs_conv_nonequil.pkl", "wb") as f:
    pickle.dump(dgs_conv_nonequil, f)

In [ ]:
import a3fe as a3
import pickle
import numpy as np

with open("final_analysis/costs.pkl", "rb") as f:
    costs = pickle.load(f)


# Convert fractions to GPU hours. For the unequilibrated leg, do this by multiplying
# by first converting the fracts to simulation times, then multipling these by the 
# relative simulation cost * the simulation cost reference of 0.21 hr / ns
REF_COST = 0.21 # GPU hours per ns
ligs = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}


REF_COST = 0.21 # GPU hours per ns

adaptive_paths = {
    "T4L": "t4l",
    "MIF": "mif_180_anti",
    "MDM2-PIP2": "mdm2_pip2_short",
    "PDE2A": "pde2a_p10",
    "MDM2-Nutlin": "mdm2_nutlin_notprot",
}

# Dict mapping directory names to calculation names
non_adapt_paths = {
    "t4l_0.2": "../non_adaptive_final/t4l_200ps",
    "t4l_6": "../non_adaptive_final/t4l_5000ps",
    "t4l_30": "../non_adaptive_final/t4l_30000ps",
    "mif_0.2": "../non_adaptive_final/mif_180_anti_200ps",
    "mif_6": "../non_adaptive_final/mif_180_anti_5000ps",
    "mif_30": "../non_adaptive_final/mif_180_anti_30000ps",
    "mdm2_nut_0.2": "../non_adaptive_final/mdm2_nutlin_notprot_200ps",
    "mdm2_nut_6": "../non_adaptive_final/mdm2_nutlin_notprot_5000ps",
    "mdm2_nut_30": "../non_adaptive_final/mdm2_nutlin_notprot_30000ps",
    "mdm2_pip_0.2": "../non_adaptive_final/mdm2_pip2_short_200ps",
    "mdm2_pip_6": "../non_adaptive_final/mdm2_pip2_short_5000ps",
    "mdm2_pip_30": "../non_adaptive_final/mdm2_pip2_short_30000ps",
    "pde_0.2": "../non_adaptive_final/pde2a_p10_200ps",
    "pde_6": "../non_adaptive_final/pde2a_p10_5000ps",
    "pde_30": "../non_adaptive_final/pde2a_p10_30000",
}

non_adapt_ligs = {
    "T4L": {"0.2 ns": "t4l_0.2", "6 ns": "t4l_6", "30 ns": "t4l_30"},
    "MIF": {"0.2 ns": "mif_0.2", "6 ns": "mif_6", "30 ns": "mif_30"},
    "MDM2-Nutlin": {"0.2 ns": "mdm2_nut_0.2", "6 ns": "mdm2_nut_6", "30 ns": "mdm2_nut_30"},
    "MDM2-PIP2": {"0.2 ns": "mdm2_pip_0.2", "6 ns": "mdm2_pip_6", "30 ns": "mdm2_pip_30"},
    "PDE2A": {"0.2 ns": "pde_0.2", "6 ns": "pde_6", "30 ns": "pde_30"},
}

calc_structure = {"bound":{"restrain":a3.StageType.RESTRAIN, "discharge":a3.StageType.DISCHARGE, "vanish":a3.StageType.VANISH}, "free": {"discharge":a3.StageType.DISCHARGE, "vanish":a3.StageType.VANISH}}

gpu_times = {}
for lig in adaptive_paths:
    gpu_times[lig] = {"Adaptive":0, "0.2 ns":0,"6 ns":0, "30 ns":0}
    for sim_label in gpu_times[lig]:
        calc_path = adaptive_paths[lig] if sim_label == "Adaptive" else non_adapt_paths[non_adapt_ligs[lig][sim_label]]
        calc = a3.Calculation(base_dir = calc_path, stream_log_level=logging.CRITICAL)
        for leg in calc.legs:
            cost = REF_COST * costs[lig][str(leg.leg_type)]
            gpu_hr = cost*leg.tot_simtime
            gpu_times[lig][sim_label] += gpu_hr
        calc._close_logging_handlers()
        del(calc)

    # Pickle the current data
    with open("final_analysis/gpu_times.pkl", "wb") as f:
        pickle.dump(gpu_times, f)
